In [ ]:
import numpy as np
import cvxpy as cp

from OPF_Tools.result import *

def runOPF(case, generation, T=1, robust=False  type=0, verb=False, solver=None, **kwargs):
    '''Find the solution to the OPF specified.
    The solver uses a sparse representation of the problem

    Type defines the type of relaxation used. These are 
    0 (default) -> SDR
    1 -> SOCR
    2 -> TCR
    3 -> STCR

    verb invokes the verbose option in cvxpy

    Returns:
        RunResult instance with the optimal values
        -> None if optimizer does not converge 
    '''
    # load data from case
    n = case.N
    baseMVA = case.mva
    Load_data = case.loadData
    Gen_data = case.genData
    Costs_data = case.cost
    Y = case.adj
    Sij = case.smax
    v_lim = case.vlim
    lines = case.getLines()

    renew = []


    #Voltage matrix
    V = [cp.Variable((n,n), hermitian=True) for _ in range(T)]

    # power transfer variables
    pij = cp.Variable((len(lines), T))
    pji = cp.Variable((len(lines), T))
    qij = cp.Variable((len(lines), T))
    qji = cp.Variable((len(lines), T))

    # total generation variable
    pi_g = cp.Variable((n, T))
    qi_g = cp.Variable((n, T))
    renew_real = cp.Variable((len(renew), T))
    renew_react = cp.Variable((len(renew), T))

    avgs = []
    stds = []

    diesel_real = cp.Variable((n, T))
    diesel_react = cp.Variable((n, T))
    sot = cp.Variable((n, T))
    pow_out = cp.Variable((n, T))
    pow_in = cp.Variable((n, T))
    max_out = 20
    max_in = 20
    max_charge = 100
    eta_in = 0.95
    eta_out = 0.95
    initial_sot = np.ones(n)*75



    # Define constraints
    constraints=[]

    constraints += [pow_out >= 0, pow_in >= 0, sot >= 0]
    constraints += [pow_out <= max_out, pow_in <= max_in, sot <= max_charge]

    #Constraints on active and reactive generation (min-max)
    constraints += [diesel_real >= Gen_data[:,1], diesel_real <= Gen_data[:,0]]
    constraints += [diesel_react >= Gen_data[:,3], diesel_react <= Gen_data[:,2]]

    batteries = []
    # Calculate the sum of all inbound power flows to each bus
    for t in range(T):

        if robust:
            constraints += [cp.norm((renew_real[:,t]-avgs)/stds) <= 0]

        for i in range(n) :  
            psum = 0
            qsum = 0
            for line in range(len(lines)):
                start, end = lines[line]
                if start == i:
                    psum += pij[line, t]
                    qsum += qij[line, t]
                elif end == i:
                    psum += pji[line, t]
                    qsum += qji[line, t]

            # Sum pij = pi
            if t>0 and i in batteries:
                constraints += [sot[i, t] == sot[i, t-1] + (pow_in[i, t]*eta_in-pow_out[i, t]*eta_out)]
            else:
                constraints += [sot[i, t] == initial_sot]
            constraints += [psum == pi_g[i, t]+pow_in[i, t]-pow_out[i, t]-Load_data[i,0, t]]
            # Sum qij = qi 
            constraints += [qsum == qi_g[i, t]-Load_data[i,1, t]]

            

            # Voltage limits
            constraints+=[cp.real(V[t][i,i])>= (v_lim[i,1])**2, cp.real(V[t][i,i]) <= (v_lim[i,0])**2]
            constraints += [cp.imag(V[t][i,i]) == 0]

        # Power flow equations (sparse representation)
        for line in range(len(lines)):
            i, j = lines[line]
                
            #Powerflow
            constraints+=[pij[line, t] + 1j*qij[line, t]==(V[t][i,i]-V[t][i,j])*np.conjugate(Y[i,j])] 
            constraints+=[pji[line, t] + 1j*qji[line, t]==(V[t][j,j]-V[t][j,i])*np.conjugate(Y[j,i])] 
            
            if not Sij[i,j] == 0:
            #Apparent power capacity S_bar
                constraints+=[cp.square(pij[line, t])+cp.square(qij[line, t])<=cp.square(Sij[i,j])]
                constraints+=[cp.square(pji[line, t])+cp.square(qji[line, t])<=cp.square(Sij[j,i])]
                
            
        # SDR problem
        if type == 0:
            constraints += [V[t] >> 0.]

        # SOCR problem
        if type == 1:
            for line in range(len(lines)):
                i, j = lines[line]
                constraints+=[cp.norm(cp.hstack([2*V[t][i,j],(V[t][i,i]-V[t][j,j])])) <= cp.real(V[t][i,i]+V[t][j,j])]
                constraints+=[cp.norm(cp.hstack([2*V[t][j,i],(V[t][j,j]-V[t][i,i])])) <= cp.real(V[t][j,j]+V[t][i,i])]

        # TCR problem
        if type == 2:
            v = cp.Variable((n), complex=True)
            constraints += [cp.real(V[0,0]) <= (v_lim[0,0]+v_lim[0,1])*cp.real(v[0])-v_lim[0,0]*v_lim[0,1]]
            constraints += [cp.imag(v[0]) == 0]
            for i in range(n):
                for j in range(n):
                    line1 = cp.hstack([1, cp.conj(v[i]), cp.conj(v[j])])
                    line2 = cp.hstack([v[i], V[i,i], V[i,j]])
                    line3 = cp.hstack([v[j], cp.conj(V[i,j]), V[j,j]])
                    mat = cp.vstack([line1, line2, line3])
                    constraints += [mat >> 0.]
        
        # STCR problem
        if type == 3:
            for i in range(n):
                for j in range(n):
                    line1 = cp.hstack([V[0,0], V[0,i], V[0,j]])
                    line2 = cp.hstack([cp.conj(V[0,i]), V[i,i], V[i,j]])
                    line3 = cp.hstack([cp.conj(V[0,j]), cp.conj(V[i,j]), V[j,j]])
                    mat = cp.vstack([line1, line2, line3])
                    constraints += [mat >> 0.]

    # Define costs
    Costs = 0
    #for i in range(n): 
    #    c0 = Costs_data[i][2]
    #    c1 = Costs_data[i][1]
    #    c2 = Costs_data[i][0]
    #    if c1 > 0.0: # Bus has a generator installed
    #        Costs += c0+c1*pi_g[i]*baseMVA+c2*cp.square(pi_g[i]*baseMVA)
    
    prob = cp.Problem(cp.Minimize(Costs), constraints)
    if solver is not None:
        prob.solve(verbose=verb, solver=solver, max_iters=max_iters)
    else:
        prob.solve(verbose=verb)
    ans = RunResult()
    try:
        ans.setAll(pi_g.value*baseMVA, qi_g*baseMVA, V.value, prob.value)
    except TypeError:
        return None
    return ans